Encode with the scaled data


In [ ]:
#!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.7-cp37-cp37m-linux_x86_64.whl

In [1]:
import torch
import numpy as np
import torch.nn as nn
from sklearn.decomposition import PCA

# import torch_xla
# import torch_xla.core.xla_model as xm
# import torch_xla.distributed.parallel_loader as pl
# import torch_xla.distributed.xla_multiprocessing as xmp

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# unscaled = np.load('/content/drive/MyDrive/Autoencoding/Unscaled_data_sorted.npy')
# train_dataset = unscaled[unscaled[:,0] < 2.0] ## taking the distence of relevance

# pca_whiten = PCA(whiten=True)

# train_dataset = pca_whiten.fit_transform(train_dataset)

# train_dataset = torch.from_numpy(train_dataset)

In [4]:
train_dataset = torch.load('/content/drive/MyDrive/1780_to_4047_files/xyz_traj_whitten.pt')

In [5]:
def lin_block(in_dim,out_dim,bias=True):
    layers = [nn.Linear(in_dim,out_dim,bias= True),
              #nn.BatchNorm2d(out_dim),
              nn.LeakyReLU(negative_slope=0.2,inplace=True)]
    return nn.Sequential(*layers)

In [6]:
hidden_layers = [256,128,64]
in_dim = 306
lat_dim = 4

In [7]:
class Resnet9(nn.Module):
      def __init__(self, in_dim,lat_dim,hidden_layers):
        super().__init__()
        
        # Encode
        
        self.lin1 = lin_block(in_dim,hidden_layers[0])     # 306 * 256   
        self.lin2 = lin_block(hidden_layers[0],hidden_layers[1])  # 256 * 128
        self.lin3 = lin_block(hidden_layers[1],hidden_layers[2])  # 128 * 64
        
        self.res1 = nn.Sequential(lin_block(hidden_layers[2], hidden_layers[2]),  # 64 * 64
                                  lin_block(hidden_layers[2], hidden_layers[2]))  # 64 *64
        
        # last layer with on activation function
        self.lin4 = nn.Linear(hidden_layers[2],lat_dim)  # 64 *4
        
        
         # Decode
        
        self.lin5 = lin_block(lat_dim,hidden_layers[2])            # 4 * 64
        self.lin6 = lin_block(hidden_layers[2],hidden_layers[1])   # 64 * 128
        self.res2 = nn.Sequential(lin_block(hidden_layers[1], hidden_layers[1]),  # 128 *128  
                                  lin_block(hidden_layers[1], hidden_layers[1]))
        self.lin7 = lin_block(hidden_layers[1],hidden_layers[0])     # 128 * 256
        self.lin8 = nn.Linear(hidden_layers[0],in_dim)               # 256 * 306
        
        
      def encode(self,in_data):
          out = self.lin1(in_data.float())
          out = self.lin2(out)
          out = self.lin3(out)
          out = self.res1(out)+out
          out = self.lin4(out)
          return out
           
      def decode(self,lat_data):
          out = self.lin5(lat_data.float())
          out = self.lin6(out)
          out = self.res2(out)+out
          out = self.lin7(out)
          out = self.lin8(out)
          return out
      
        

In [8]:
#device=torch.device('cuda')
Encoder = Resnet9(in_dim,lat_dim,hidden_layers) #.to(device)

In [9]:
Encoder.load_state_dict(torch.load('/content/drive/MyDrive/1780_to_4047_files/model_state_dict/encoder_state_dict_resnet.pth'))

<All keys matched successfully>

In [25]:
whitten_data = torch.load('/content/drive/MyDrive/1780_to_4047_files/xyz_traj_whitten.pt')

In [24]:
ls /content/drive/MyDrive/1780_to_4047_files/*pt

/content/drive/MyDrive/1780_to_4047_files/xyz_traj_whitten.pt


In [42]:
torch.min(whitten_data[:,302])

tensor(-0.3259, dtype=torch.float64)

In [21]:
data = torch.zeros(128,306)*2

In [ ]:
Encoder.encode(data)

In [23]:
for param_tensor in Encoder.state_dict():
    print(param_tensor, "\t", Encoder.state_dict()[param_tensor])

lin1.0.weight 	 tensor([[-2.4869e-34, -2.0403e-34,  2.1307e-34,  ..., -3.0189e-34,
         -8.2130e-34,  6.1389e-34],
        [-1.2040e-34,  1.0168e-33, -6.6314e-34,  ..., -1.9911e-34,
         -7.0723e-34,  1.3364e-34],
        [-3.4628e-35, -8.4684e-34, -7.6050e-34,  ..., -8.2133e-34,
         -8.2808e-35, -1.1306e-34],
        ...,
        [-3.0979e-34,  4.5380e-34,  5.8032e-34,  ...,  2.4311e-34,
         -8.3770e-34, -2.7139e-35],
        [ 2.8578e-34, -8.5910e-34, -8.0125e-34,  ...,  8.4044e-34,
          5.1702e-34,  9.7892e-36],
        [-4.9388e-34,  1.6758e-34, -6.0723e-35,  ...,  4.0940e-34,
          8.2155e-34, -1.3698e-34]])
lin1.0.bias 	 tensor([ 8.5133e-35, -1.4710e-34,  7.9644e-34, -4.1307e-34, -1.3284e-34,
         9.5964e-35, -2.5499e-34,  9.6504e-34, -3.6873e-34,  1.5698e-34,
        -6.7820e-34,  8.4449e-34,  4.8232e-34, -3.6583e-35, -3.4350e-34,
         8.4577e-34,  1.8015e-34, -2.1054e-35, -5.7781e-34,  3.8096e-34,
        -1.6338e-35,  3.6066e-34,  8.1268e-34,

In [ ]:
train_dataset.shape

torch.Size([2252250, 306])

In [ ]:
2252250/2250

1001.0

In [ ]:
x= int(2252250/1001)
print(x)

2250


In [ ]:
import gc
gc.collect()

310

In [ ]:
from tqdm.notebook  import tqdm

In [ ]:
#who_ls

In [ ]:
#lat1 = make_2_dim(encoder.encode(make_4_dim(train_dataset.to(device))))

In [ ]:
%%time
#latent = []
for i in tqdm(range(1001)):
    lat1 = Encoder.encode((train_dataset[(i)*x:((i+1)*x)])) # use .to(device) for GPU training
    #z_t = VDE.add_noise(train_dataset[(i-1)*x:(i*x)].to(device)) 
    torch.save(lat1,'lat'+str(i)+'.pt')
    #latent.append(lat1)
    del lat1 
    gc.collect()


CPU times: user 4min 1s, sys: 963 ms, total: 4min 2s
Wall time: 2min 29s


In [ ]:
import glob

**To sort numerically**

In [ ]:
from natsort import natsort_keygen, ns
natsort_key1 = natsort_keygen(key=lambda y: y.lower())

In [ ]:
file = sorted(glob.glob('lat*'),key=natsort_key1)

In [ ]:
file

['lat0.pt',
 'lat1.pt',
 'lat2.pt',
 'lat3.pt',
 'lat4.pt',
 'lat5.pt',
 'lat6.pt',
 'lat7.pt',
 'lat8.pt',
 'lat9.pt',
 'lat10.pt',
 'lat11.pt',
 'lat12.pt',
 'lat13.pt',
 'lat14.pt',
 'lat15.pt',
 'lat16.pt',
 'lat17.pt',
 'lat18.pt',
 'lat19.pt',
 'lat20.pt',
 'lat21.pt',
 'lat22.pt',
 'lat23.pt',
 'lat24.pt',
 'lat25.pt',
 'lat26.pt',
 'lat27.pt',
 'lat28.pt',
 'lat29.pt',
 'lat30.pt',
 'lat31.pt',
 'lat32.pt',
 'lat33.pt',
 'lat34.pt',
 'lat35.pt',
 'lat36.pt',
 'lat37.pt',
 'lat38.pt',
 'lat39.pt',
 'lat40.pt',
 'lat41.pt',
 'lat42.pt',
 'lat43.pt',
 'lat44.pt',
 'lat45.pt',
 'lat46.pt',
 'lat47.pt',
 'lat48.pt',
 'lat49.pt',
 'lat50.pt',
 'lat51.pt',
 'lat52.pt',
 'lat53.pt',
 'lat54.pt',
 'lat55.pt',
 'lat56.pt',
 'lat57.pt',
 'lat58.pt',
 'lat59.pt',
 'lat60.pt',
 'lat61.pt',
 'lat62.pt',
 'lat63.pt',
 'lat64.pt',
 'lat65.pt',
 'lat66.pt',
 'lat67.pt',
 'lat68.pt',
 'lat69.pt',
 'lat70.pt',
 'lat71.pt',
 'lat72.pt',
 'lat73.pt',
 'lat74.pt',
 'lat75.pt',
 'lat76.pt',
 'lat77.p

In [ ]:
X = []
for f in file:
    X.append(torch.load(f))

X1 = torch.tensor(torch.cat(X))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """


In [ ]:
X1.shape

torch.Size([2252250, 4])

In [ ]:
rm -r lat*

In [ ]:
torch.save(X1,'encoded_latent_4D.pt')

In [ ]:
mv encoded_latent_4D.pt /content/drive/MyDrive/1780_to_4047_files/model_state_dict

In [ ]:
X1.size()

torch.Size([2252250, 4])

In [ ]:
#import pyemma

In [ ]:
!mv latent_not_sampled.pt /content/drive/MyDrive/VDE_with_scaling/scale_0.1